### Model training runs for testing models trained with two neighboring stations

The results can be found [here](https://app.neptune.ai/o/data-mining-team2/org/model-tests/runs/details?viewId=995f0563-1e7a-4513-b399-85c5addd3334)


In [1]:
import tensorflow as tf


import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback

import os

import numpy as np
import pandas as pd
from window_generator import WindowGenerator

from run_automation_utils import Params, init_model

from typing import Dict


In [2]:
def prep_window_generator(station_id, second_station_id=None):
    df = pd.read_feather(f"../../data/pollution/processed/interpolated/pm10/{station_id}.feather")
    df.set_index("timestamp", inplace=True)

    if second_station_id is not None:
        df2 = pd.read_feather(f"../../data/pollution/processed/interpolated/pm10/{second_station_id}.feather")
        df2.set_index("timestamp", inplace=True)
        df = pd.concat([df, df2])

    timestamp_s = df.index.map(pd.Timestamp.timestamp)

    day = 24 * 60 * 60
    year = (365.2425) * day

    df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
    df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
    df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
    df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
    df.reset_index(inplace=True)

    # Convert to radians.
    wd_rad = df.pop("winddirection_10m") * np.pi / 180
    df["winddirection_10m_sin"] = np.sin(wd_rad)
    df["winddirection_10m_cos"] = np.cos(wd_rad)

    return WindowGenerator(input_width=24 * 7, label_width=24, df=df.copy(), label_columns=["pm10"])


In [3]:
window_generator_530 = prep_window_generator(530)
window_generator_538 = prep_window_generator(538)
window_generator_530_538 = prep_window_generator(530, 538)


In [4]:
run_params: Dict[str, Params] = {}
run_params["station_530_with_lambda"] = Params(window_generator_530, lstm_layer_count=2, lstm_sizes=[64, 64])
run_params["station_538_with_lambda"] = Params(window_generator_538, lstm_layer_count=2, lstm_sizes=[64, 64])
run_params["station_530_and_538_with_lambda"] = Params(
    window_generator_530_538, lstm_layer_count=2, lstm_sizes=[64, 64]
)


In [5]:
import logging

logger = logging.getLogger("run_logger")
fileHandler = logging.FileHandler("run.logs.txt")
logger.addHandler(fileHandler)


In [6]:
for run_id, params in run_params.items():
    logger.error("Start run: " + run_id)
    model = init_model(params)

    loss = tf.keras.losses.MeanSquaredError()
    if params.loss_function == "huber":
        loss = tf.keras.losses.huber
    model.compile(loss=loss, optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()])

    name = params.to_name(100)
    temp_name = name
    append_index = 0
    while os.path.exists("../models/" + temp_name):
        append_index += 1
        temp_name = name + "_" + str(append_index)
    if append_index != 0:
        name += "_" + str(append_index)

    run = neptune.init_run(
        project="data-mining-team2/model-tests",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4Mzg2ZWZmYi05YzRlLTQ3ODYtOWE1NC1mNDM4OTM1ZjNlOTkifQ==",
        custom_run_id=run_id,
        name=name,
        source_files=["./model_tests_neighboring_stations.ipynb"],
    )

    run["parameters"] = params.to_dict()

    neptune_cbk = NeptuneCallback(run=run, base_namespace="training")

    try:
        for epoch_count_factor in range(1, 5):
            epoch_count = 50 * epoch_count_factor
            history = model.fit(
                params.window_generator.train,
                epochs=50,
                validation_data=params.window_generator.val,
                # callbacks=[neptune_cbk],                                                  # commented out so any further runs aren't tracked from neptune
            )

            eval_metrics = model.evaluate(params.window_generator.test, verbose=0)
            for j, metric in enumerate(eval_metrics):
                run[f"eval/epoch_{epoch_count}/{model.metrics_names[j]}"] = metric

            name = params.to_name(epoch_count)
            temp_name = name
            append_index = 0
            while os.path.exists("../models/" + temp_name):
                append_index += 1
                temp_name = name + "_" + str(append_index)
            if append_index != 0:
                name += "_" + str(append_index)

            model.save("../models/" + name, include_optimizer=False)

            run["model_names/" + str(epoch_count) + "epochs"] = name
    except Exception as e:
        logger.error(e)

        pass
    run.stop()
    logger.error("Finished run: " + run_id)


e:\FHBielefeld\Master\DataMining\feinstaubprojekt-polen\.env\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


https://app.neptune.ai/data-mining-team2/model-tests/e/MOD-11
Epoch 1/50


e:\FHBielefeld\Master\DataMining\feinstaubprojekt-polen\.env\lib\site-packages\neptune\common\warnings.py:62: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'window_generator.WindowGenerator'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  warnings.warn(


707/707 [==============================] - 12s 11ms/step - loss: 504.2591 - mean_absolute_error: 14.6754 - val_loss: 413.0365 - val_mean_absolute_error: 13.4832
Epoch 2/50
707/707 [==============================] - 7s 10ms/step - loss: 415.9999 - mean_absolute_error: 13.1960 - val_loss: 407.2528 - val_mean_absolute_error: 13.3157
Epoch 3/50
707/707 [==============================] - 7s 10ms/step - loss: 415.6514 - mean_absolute_error: 13.1733 - val_loss: 401.9151 - val_mean_absolute_error: 13.0482
Epoch 4/50
707/707 [==============================] - 7s 10ms/step - loss: 409.0658 - mean_absolute_error: 13.0411 - val_loss: 401.8313 - val_mean_absolute_error: 13.3793
Epoch 5/50
707/707 [==============================] - 7s 10ms/step - loss: 403.4806 - mean_absolute_error: 12.9401 - val_loss: 395.5782 - val_mean_absolute_error: 13.0913
Epoch 6/50
707/707 [==============================] - 7s 10ms/step - loss: 398.3820 - mean_absolute_error: 12.8355 - val_loss: 385.7459 - val_mean_absolute

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_50_epochs_2lstm-layer_64,64_dropout-0-4_3\assets


...

Epoch 1/50
1389/1389 [==============================] - 27s 19ms/step - loss: 110.5382 - mean_absolute_error: 6.7859 - val_loss: 215.0183 - val_mean_absolute_error: 9.1398
Epoch 2/50
1389/1389 [==============================] - 26s 19ms/step - loss: 112.4576 - mean_absolute_error: 6.8589 - val_loss: 218.2057 - val_mean_absolute_error: 9.1522
Epoch 3/50
1389/1389 [==============================] - 27s 19ms/step - loss: 113.6685 - mean_absolute_error: 6.8921 - val_loss: 215.2026 - val_mean_absolute_error: 9.1138
Epoch 4/50
1389/1389 [==============================] - 27s 19ms/step - loss: 115.9340 - mean_absolute_error: 6.9455 - val_loss: 211.8120 - val_mean_absolute_error: 9.0762
Epoch 5/50
1389/1389 [==============================] - 27s 19ms/step - loss: 111.6021 - mean_absolute_error: 6.8288 - val_loss: 215.9768 - val_mean_absolute_error: 9.1194
Epoch 6/50
1389/1389 [==============================] - 27s 19ms/step - loss: 111.1125 - mean_absolute_error: 6.8123 - val_loss: 213.7837 - 

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_200_epochs_2lstm-layer_64,64_dropout-0-4_5\assets


INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_200_epochs_2lstm-layer_64,64_dropout-0-4_5\assets


Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/data-mining-team2/model-tests/e/MOD-13/metadata


ERROR:run_logger:Finished run: station_530_and_538_with_lambda


#### Load the models and plot the same windows from every window generator with prediction from every model


In [14]:
model_double = tf.keras.models.load_model(
    "../models/cnn_lstm_7days_history_1days_pred_200_epochs_2lstm-layer_64,64_dropout-0-4_5"
)
model_530 = tf.keras.models.load_model(
    "../models/cnn_lstm_7days_history_1days_pred_200_epochs_2lstm-layer_64,64_dropout-0-4_3"
)
model_538 = tf.keras.models.load_model(
    "../models/cnn_lstm_7days_history_1days_pred_200_epochs_2lstm-layer_64,64_dropout-0-4_4"
)


In [5]:
window_generator_530 = prep_window_generator(530)
window_generator_538 = prep_window_generator(538)


In [40]:
window_generator_530.plot(model_double, offset=100)


In [41]:
window_generator_530.plot(model_530, offset=100)


In [42]:
window_generator_530.plot(model_538, offset=100)


In [49]:
window_generator_538.plot(model_double, offset=10)


In [48]:
window_generator_538.plot(model_530, offset=10)


In [47]:
window_generator_538.plot(model_538, offset=10)


In [50]:
model_530.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 168, 11)          23        
 n)                                                              
                                                                 
 lambda (Lambda)             (None, 5, 11)             0         
                                                                 
 conv1d (Conv1D)             (None, 5, 128)            7168      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 128)           0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 2, 128)           512       
 ormalization)                                                   
                                                        

In [61]:
test = np.array([[[1, 1], [1, 2], [1, 3]], [[2, 1], [2, 2], [2, 3]], [[3, 1], [3, 2], [3, 3]]])


In [58]:
test.shape


(3, 3, 2)